<h1><center><b>Tecnologias de Processamento de Dados - 2019/2020<b></center></h1>
<h2><center>Fase 2</center></h2> 
<h2><center>Grupo 21</center><h2>
<table>
  <tr>
    <th><h3><b>Nome</b></h3></th>
    <th><h3><b>Número</b></h3></th>
    <th><h3><b>Horas</b></h3></th>
  </tr>
  <tr>
    <td><h3>Diogo Silva</h3></td>
    <td><h3>54131</h3></td>
    <td><h3>22</h3></td>
  </tr>
  <tr>
    <td><h3>Nuno Silva</h3></td>
    <td><h3>55109</h3></td>
    <td><h3>22</h3></td>
  </tr>
  <tr>
    <td><h3>Pedro Russo</h3></td>
    <td><h3>55101</h3></td>
    <td><h3>22</h3></td>
  </tr>
</table>

<h2>Imports</h2>

In [0]:
import pandas as pd
from datetime import datetime
import calendar
from geopy.geocoders import Nominatim
import json
import hashlib
import psycopg2
import time

<h2> 1. Dimensions and facts tables of the data warehouse </h2>


*   *Define and model them in SQL*
*   *Create the dimensions and  facts tables in the DBMS (postgreSQL)* 



In [0]:
def create_tables():
  commands = (
    """
    create TABLE DIMAircraft(
      AircraftID varChar(32) PRIMARY KEY,
      AircraftCategory text,
      RegistrationNumber text,
      Make text,
      Model text,
      AmateurBuilt varChar(8),
      NumberOfEngines integer,
      EngineType text
    )""","""
    create TABLE DIMAccident(
      AccidentID varChar(32) PRIMARY KEY,
      AircraftDamage text,
      BroadPhaseOfFlight text,
      WeatherCondition text
    )""","""
    create TABLE DIMLocation(
      LocationID varChar(32) PRIMARY KEY,
      Latidude text,
      Longitude text,
      LatLong text,
      Location text,
      Country text,
      State text,
      County text,
      City text,
      Road text,
      AirportCode text,
      AirportName text,
      ArrivalAirport text,
      DepartureAirport text
    )""","""
    create TABLE DIMDate(
      DateID varChar(32) PRIMARY KEY,
      Date date,
      FullDateDescription date,
      DayOfWeek varChar(10),
      CalendarMonth varChar(10),
      CalendarYear integer,
      FiscalYearMonth varChar(8),
      Semester integer,
      Quarter integer,
      Trimester integer,
      WeekdayIndicator varChar(8)
    )""","""
    create TABLE Fact(
      TotalFatalInjuries integer,
      TotalSeriousInjuries integer,
      TotalMinorInjuries integer,
      TotalUninjured integer,
      FKAirCraft varchar(32),
      FKAccident varchar(32),
      FKLocation varchar(32),
      FKDate varchar(32),
      FOREIGN KEY(FKAirCraft)
        REFERENCES DIMAircraft(AircraftID)
        ON update CASCADE ON delete CASCADE,
      FOREIGN KEY(FKAccident)
        REFERENCES DIMAccident(AccidentID)
        ON update CASCADE ON delete CASCADE,
      FOREIGN KEY(FKLocation)
        REFERENCES DIMLocation(LocationID)
        ON update CASCADE ON delete CASCADE,
      FOREIGN KEY(FKDate)
        REFERENCES DIMDate(DateID)
        ON update CASCADE ON delete CASCADE
    )"""
  )
  connection = None
  try:
    connection = psycopg2.connect(user = "kpswytoq",
                                password = "ds8b6HdwJ11N6EE2W-iGowWFTrsnwGch",
                                host = "kandula.db.elephantsql.com",
                                port = "5432",
                                database = "kpswytoq")
    cur = connection.cursor()
    for command in commands:
      cur.execute(command)
    cur.close()
    connection.commit()
    print("Tables created")
  except (Exception, psycopg2.DatabaseError) as error:
    print(error)
  finally:
    if connection is not None:
      connection.close()

create_tables()



*   *Identify hierarchies and fact granularity*




**DimDate:** Date > Full Date Description > Cakendar Year > Calendar Month > Fiscal Year-Month > Semester > Trimester Weekday indicator > Day of the week 

**DimLocation:** Country > Location > State > County > City > Road

**DimAircraft:** Make >  AircraftCategory  > Model

**Granularidade dos factos:** Uma linha por evento de um acidente com uma aeronave especifica, num certo local e numa data especifica.





<h2> 2. ETL workflow </h2>

*   *Identify all data sources for all dimensions. Add URL links to all data that should be available. If not public data, point to dropbox files, Google drive, or whatever*

Kaggle Dataset: https://www.kaggle.com/khsamaha/aviation-accident-database-synopses

Repositório do GitHub: https://github.com/TryHarderOfGit/TPD

Biblioteca geopy: https://pypi.org/project/geopy/

*   *For each dimension show the code used for modeling, filtering and inserting data*

<h2>DimAccident</h2>



In [0]:
def dimAccident(row, array, cursor):
  AircraftDamage = row['Aircraft.Damage']
  BroadPhaseOfFlight = row['Broad.Phase.of.Flight']
  WeatherCondition = row['Weather.Condition']
  Accident = {
    'AccidentID' : None,
    'AircraftDamage' : AircraftDamage,
    'BroadPhaseOfFlight' : BroadPhaseOfFlight,
    'WeatherCondition' : WeatherCondition
  }
  AccidentID = ''
  for key, value in Accident.items():
    if (key != 'AccidentID'):
      AccidentID = AccidentID + str(value)
  md5 = hashlib.md5()
  md5.update(AccidentID.encode('utf-8'))
  AccidentID = md5.hexdigest()
  Accident['AccidentID'] = AccidentID
  if Accident not in array:
    array.append(Accident)
    cursor.execute("INSERT INTO \"public\".\"dimaccident\" values(%s,%s,%s,%s)",(
      AccidentID,
      AircraftDamage,
      BroadPhaseOfFlight,
      WeatherCondition
    ))
  return AccidentID

Amostra de 5 resultados da dimensão Accident no PostgreSQL

![https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimAccident.png](https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimAccident.png)



---


<h2>DimAircraft</h2>

In [0]:
def dimAircraft(row, array, cursor):
  AircraftCategory = row['Aircraft.Category']
  RegistrationNumber = row['Registration.Number']
  Make = row['Make']
  Model = row['Model']
  AmateurBuilt = row['Amateur.Built']
  NumberOfEngines = row['Number.of.Engines']
  EngineType = row['Engine.Type']
  if NumberOfEngines == 'Unknown':
    NumberOfEngines = -1
  Aircraft = {
    'AircraftID' : None,
    'AircraftCategory' : AircraftCategory,
    'RegistrationNumber' : RegistrationNumber,
    'Make' : Make,
    'Model' : Model,
    'AmateurBuilt' : AmateurBuilt,
    'NumberOfEngines' : NumberOfEngines,
    'EngineType' : EngineType
  }
  AircraftID = ''
  for key, value in Aircraft.items():
    if (key != 'AircraftID'):
      AircraftID = AircraftID + str(value)
  md5 = hashlib.md5()
  md5.update(AircraftID.encode('utf-8'))
  AircraftID = md5.hexdigest()
  Aircraft['AircraftID'] = AircraftID
  if Aircraft not in array:
    array.append(Aircraft)
    cursor.execute("INSERT INTO \"public\".\"dimaircraft\" values(%s,%s,%s,%s,%s,%s,%s,%s)",(
      md5.hexdigest(),
      AircraftCategory,
      RegistrationNumber,
      Make,
      Model,
      AmateurBuilt,
      NumberOfEngines,
      EngineType
    ))
  return AircraftID

Amostra de 5 resultados da dimensão Aircraft no PostgreSQL

![https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimAircraft.png](https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimAircraft.png)



---


<h2>DimLocation</h2>

In [0]:
def dimLocation(row, array, cursor):
  Latitude = row['Latitude']
  Longitude = row['Longitude']
  LatLong = 'Unknown'
  LocationX = row['Location']
  Country = row['Country']
  State = 'Unknown'
  County = 'Unknown'
  City = 'Unknown'
  Road = 'Unknown'
  BroadPhaseOfFlight = row['Broad.Phase.of.Flight']
  AirportCode = row['Airport.Code']
  AirportName = row['Airport.Name']
  ArrivalAirport = 'Unknown'
  DepartureAirport = 'Unknown'
  if (Latitude != 'Unknown' and Longitude != 'Unknown'):
      LatLong = str(Latitude) + "," + str(Longitude)
  if AirportName == 'N/A':
    AirportName = 'Unknown'
  geolocation = None
  if LatLong != 'Unknown':
    geolocation = geolocator.reverse(LatLong, language='en')
    geolocationJson = json.dumps(geolocation.raw, indent=4)
    geolocationObj = json.loads(geolocationJson)
    if 'address' in geolocationObj:
      address = geolocationObj['address']
      if 'country' in address:
        Country = address['country']
      if 'state' in address:
        State = address['state']
      if 'county' in address:
        County = address['county']
      if 'aerodrome' in address:
        AirportName = address['aerodrome']
      if 'city' in address:
        City = address['city']
      if 'road' in address:
        Road = address['road']
  if Country == 'United States':
    Country = 'United States of America'
  if BroadPhaseOfFlight == 'TAKEOFF':
    DepartureAirport = AirportName
  elif BroadPhaseOfFlight == 'APPROACH' or BroadPhaseOfFlight == 'LANDING':
    ArrivalAirport = AirportName
  Location = {
    'LocationID': None,
    'Latidude': Latitude,
    'Longitude': Longitude,
    'LatLong': LatLong,
    'Location': LocationX,
    'Country': Country,
    'State': State,
    'County': County,
    'City': City,
    'Road': Road,
    'AirportCode': AirportCode,
    'AirportName': AirportName,
    'ArrivalAirport': ArrivalAirport,
    'DepartureAirport': DepartureAirport
  }
  LocationID = ''
  for key, value in Location.items():
    if (key != 'LocationID'):
      LocationID = LocationID + str(value)
  md5 = hashlib.md5()
  md5.update(LocationID.encode('utf-8'))
  LocationID = md5.hexdigest()
  Location['LocationID'] = LocationID
  if Location not in array:
    array.append(Location)
    cursor.execute("INSERT INTO \"public\".\"dimlocation\" values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(
      LocationID,
      Latitude,
      Longitude,
      LatLong,
      LocationX,
      Country,
      State,
      County,
      City,
      Road,
      AirportCode,
      AirportName,
      ArrivalAirport,
      DepartureAirport
    ))
  return LocationID

Amostra de 5 resultados da dimensão Location no PostgreSQL

![https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimLocation.png](https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimLocation.png)



---


<h2>DimDate</h2>

In [0]:
def dimDate(row, array, cursor):
  dateRow = row['Event.Date']
  dateTypeFormat = datetime.strptime(dateRow, '%Y-%m-%d')
  DateX = dateTypeFormat.strftime('%m/%d/%Y'),
  FullDateDescription = dateTypeFormat.strftime('%B %d, %Y'),
  DayOfWeek = calendar.day_name[dateTypeFormat.weekday()],
  CalendarMonth = dateTypeFormat.strftime('%B'),
  CalendarYear = dateTypeFormat.strftime('%Y'),
  FiscalYearMonth = dateTypeFormat.strftime('F%Y-%m'),
  Semester = 1
  Quarter = 1
  Trimester = 1
  WeekdayIndicator = 'Weekday'
  if 7 <= int(dateTypeFormat.strftime('%m')) <= 12:
    Semester = 2
  if 5 <= int(dateTypeFormat.strftime('%m')) <= 9:
    Quarter = 2
  if 10 <= int(dateTypeFormat.strftime('%m')) <= 12:
    Quarter = 3
  if 4 <= int(dateTypeFormat.strftime('%m')) <= 6:
    Trimester = 2
  elif 7 <= int(dateTypeFormat.strftime('%m')) <= 9:
    Trimester = 3
  elif 10 <= int(dateTypeFormat.strftime('%m')) <= 12:
    Trimester = 4
  if 5 <= dateTypeFormat.weekday() <= 6:
    weekdayIndicator = 'Weekend'
  Date = {
    'DateID': None,
    'Date': DateX,
    'FullDateDescription': FullDateDescription,
    'DayOfWeek': DayOfWeek,
    'CalendarMonth': CalendarMonth,
    'CalendarYear': CalendarYear,
    'FiscalYearMonth': FiscalYearMonth,
    'Semester': Semester,
    'Quarter': Quarter,
    'Trimester': Trimester,
    'WeekdayIndicator': WeekdayIndicator
  }
  DateID = ''
  for key, value in Date.items():
    if (key != 'DateID'):
      DateID = DateID + str(value)
  md5 = hashlib.md5()
  md5.update(DateID.encode('utf-8'))
  DateID = md5.hexdigest()
  Date['DateID'] = DateID
  if Date not in array:
    array.append(Date)
    cursor.execute("INSERT INTO \"public\".\"dimdate\" values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(
      DateID,
      DateX,
      FullDateDescription,
      DayOfWeek,
      CalendarMonth,
      CalendarYear,
      FiscalYearMonth,
      Semester,
      Quarter,
      Trimester,
      WeekdayIndicator
    ))
  return DateID

Amostra de 5 resultados da dimensão Date no PostgreSQL

![https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimDate.png](https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraDimDate.png)

*   *Describe the process for inserting facts data*


<h2>Fact table</h2>

In [0]:
def fact(row, arruay, fks, cursor):
  TotalFatalInjuries = row['Total.Fatal.Injuries']
  TotalSeriousInjuries = row['Total.Serious.Injuries']
  TotalMinorInjuries = row['Total.Minor.Injuries']
  TotalUninjured = row['Total.Uninjured']
  if TotalFatalInjuries == 'Unknown':
    TotalFatalInjuries = -1
  if TotalSeriousInjuries == 'Unknown':
    TotalSeriousInjuries = -1
  if TotalMinorInjuries == 'Unknown':
    TotalMinorInjuries = -1
  if TotalUninjured == 'Unknown':
    TotalUninjured = -1
  Fact = {
    'TotalFatalInjuries' : TotalFatalInjuries,
    'TotalSeriousInjuries' : TotalSeriousInjuries,
    'TotalMinorInjuries' : TotalMinorInjuries,
    'TotalUninjured' : TotalUninjured,
    'FKAircraft' : fks['Aircraft'],
    'FKAccident_FK' : fks['Accident'],
    'FKLocation' : fks['Location'],
    'FKDate' : fks['Date']
  }
  array.append(Fact)
  cursor.execute("INSERT INTO \"public\".\"fact\" values(%s,%s,%s,%s,%s,%s,%s,%s)",(
    TotalFatalInjuries,
    TotalSeriousInjuries,
    TotalMinorInjuries,
    TotalUninjured,
    fks['Aircraft'],
    fks['Accident'],
    fks['Location'],
    fks['Date']
  ))

Amostra de 5 resultados da tabela Fact no PostgreSQL

![https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraFact.png](https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/pgAmostraFact.png)

<h2><b>Process</b></h2>




In [0]:
def main():
  start_time = time.time()
  geolocator = Nominatim(user_agent="TPD")
  connection = None
  try:
    connection = psycopg2.connect(user = "kpswytoq",
                                password = "ds8b6HdwJ11N6EE2W-iGowWFTrsnwGch",
                                host = "kandula.db.elephantsql.com",
                                port = "5432",
                                database = "kpswytoq")
    cursor = connection.cursor()

    df = pd.read_csv('https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/dataset.csv', encoding = "ISO-8859-1")
    df = df.fillna('Unknown')

    accidentArray = []
    aircraftArray = []
    locationArray = []
    dateArray = []
    factArray = []

    for i, row in df.iterrows():
      Accident = dimAccident(row, accidentArray, cursor)
      Aircraft = dimAircraft(row, aircraftArray, cursor)
      Location = dimLocation(row, locationArray, cursor)
      Date = dimDate(row, dateArray, cursor)
      fks = {
          'Accident': Accident,
          'Aircraft': Aircraft,
          'Location': Location,
          'Date': Date
      }
      fact(row, factArray, fks, cursor)
      if i == 100:
        break
    
    accidentDF = pd.DataFrame(accidentArray)
    aircraftDF = pd.DataFrame(aircraftArray)
    locationDF = pd.DataFrame(locationArray)
    dateDF = pd.DataFrame(dateArray)
    factDF = pd.DataFrame(factArray)

    ''' display(accidentDF.head(5))
    display(aircraftDF.head(5))
    display(locationDF.head(5))
    display(dateDF.head(5))
    display(factDF.head(5)) '''

    cursor.close()
    connection.commit()
  except (Exception, psycopg2.DatabaseError) as error:
    print(error)
  finally:
    if connection is not None:
      connection.close()
    print("%s seconds" % (time.time() - start_time))

main()

<h2> Descrição de todo o processo </h2>

Começamos por carregar o dataset com a biblioteca pandas, de seguida preenchemos os campos a nulo com 'Unknow' e prosseguimos com a iteração das linhas do dataset. Para cada linha são executadas as funções de cada dimensão acima apresentadas, fazendo a sua inserção tanto nos array locais, como nas respetivas tabelas da base de dados, retornando a sua primary key, para que posteriormente estas sejam usadas como foreign key's para a inserção dos factos.



<h2> 3. Critical assessment of the work </h2>

* *Describe potential issues with the ETL procedure used*

A dimensão Location através do uso da biblioteca (geopy) para recolher informação do local do acidente através das coordenadas é o principal problema no processo ETL. Isto é, a biblioteca demora alguns milisegundos ou até mesmo segundos a retornar a informação para um ponto, causando atraso no processo em comparação com as outras dimensões. Esta biblioteca é uma mais valia para a futura análise, porque retorna dados interessantes e que não estão presentes no dataset.

* *Compare your schema to the one previously defined in phase I*

**Dimensão Date:** Foi adicionado o Full Date Description, Fiscal Year-Month, Day of week, Weekday Indicator e Quarter.
Todas estas novas adições foram efectuadas para complemetar a dimensão e fornecer um maior detalhe.
Day foi removido porque não representava informação relevante à dimensão.


**Dimensão Location:** Nesta dimensão foi adicionado o State, City, Road, Arrival Airport, Departure Airport e LatLong. As três primeiras foram adicionadas através da informação recolhida com a biblioteca do geopy, sendo que esta também preenche outros campos quando não existem no dataset, como o Airport Name e Country. Arrival Airport e Departure Airport são preenchidos de acordo com a Broad Phase of Flight e o Airport Name.


**Tabela de Factos:** Na tabela de Factos foram adicionadas as foreing keys de todas as dimensões.

<table>
  <tr>
    <th><h3>Antigo</h3></th>
    <th><h3>Atual</h3></th>
  </tr>
  <tr>
    <td><img src="https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/StarSchema_Antigo.png" alt="https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/StarSchema_Antigo.png"></td>
    <td><img src="https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/starSchema.png" alt="https://raw.githubusercontent.com/TryHarderOfGit/TPD/master/starSchema.png"></td>
  </tr>
</table> 



* *Discuss the issues for updating the data warehouse with novel data*

A inserção de novos dados no DW poderia levantar alguns problemas, como: Os dados inseridos excederem os limites dos atributos, por exemplo, onde existe atualmente só informação com "Yes", "No" ou "Unknown" no campo Amateur Build, no caso de ser inserida informação com maior comprimento que o estipulado, ocorrerão erros, portanto temos de ter em conta o que é inserido filtrando e modificando novos dados ou alterando a estrutura das tabelas; O tempo que demora a recolher informação através da biblioteca geopy, como descrito nos problemas do processo ETL acima. 